# Delhi Colonies Public Services Index

* Load in colonies dataset from Pickle file `colonies_10Aug2020.data` **[done]**
    * Create two versions of colonies, with one based on touching vs. bbox neighbors
    * Give each version a distinct name
* Import services shapefiles **[done]**
    * Make sure correct file paths exist
    * Ensure that all shapefiles are valid using `check_shapefile` function
    * Reproject shapefiles to EPSG 7760 (if needed)
* Compute all Services indices (turn into a function)
    * Compute Point Services Indices
    * Compute Roads Index (make sure that units are in kilometers)
    * Compute average of all service indices
    * Run services indices twice (on touching vs. bbox neighbors) using function
* Compute simple statistics (mean, min, max) based on colony type
    * Turn into function
    * Run on both datasets

## Import modules and set constants

In [1]:
import os
import pickle
from importlib import reload
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, box
import spatial_index_utils

In [2]:
reload(spatial_index_utils)

<module 'spatial_index_utils' from 'C:\\Users\\bwbel\\Google Drive\\slum_project\\spatial_index_python\\spatial_index_utils.py'>

In [3]:
# WGS 84 / Delhi
epsg_code = 7760

## Import Colonies and subset to have area > .009

In [4]:
with open('colonies_10Aug2020.data', 'rb') as f:
    colonies = pickle.load(f)

In [5]:
colonies.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,canal,railway,drain,barrier,nbrs_touch,nbrs_bbox,centroid,nbrs_dist_touch,nbrs_dist_bbox,ndmc_dist_km,area_km2,index,population
0,Singhola,3058,H,RV,"POLYGON Z ((1013763.588 1023721.838 0.000, 101...",False,False,True,True,[],"[5108, 5487, 5488]",POINT (1012643.931 1023967.383),[],"[(5108, 1.4513506146004875), (5487, 1.17056023...",24.721016,2.733999,0,4415.586042
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((1007997.730 1025421.961 0.000, 100...",False,True,False,True,"[2869, 5445]","[2869, 5445]",POINT (1008331.501 1025234.188),"[(2869, 0.5868185653539563), (5445, 0.31827133...","[(2869, 0.5868185653539563), (5445, 0.31827133...",27.676855,0.182393,1,3990.707763
2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((1008543.236 1022671.585 0.000, 100...",False,True,False,True,[5478],"[4270, 4273, 5478]",POINT (1009672.909 1022802.723),"[(5478, 0.661418841164336)]","[(4270, 0.8033104667179346), (4273, 0.58109669...",24.900046,1.470808,2,6849.523714
3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((1008080.674 1025132.190 0.000, 100...",False,True,False,True,"[2869, 5095, 5105, 5436, 5457]","[2869, 3963, 4243, 5092, 5095, 5105, 5436, 544...",POINT (1008902.120 1025098.259),"[(2869, 0.5676799818012491), (5095, 0.32048263...","[(2869, 0.5676799818012491), (3963, 0.70232814...",27.289733,1.150775,3,26712.165262
4,Kureni,2082,H,RV,"POLYGON Z ((1009508.695 1025281.671 0.000, 100...",False,True,False,True,"[3963, 5092, 5098, 5104, 5106, 5451]","[3963, 5092, 5095, 5098, 5104, 5106, 5451, 5478]",POINT (1009491.379 1023952.320),"[(3963, 0.6363487153623809), (5092, 1.29943169...","[(3963, 0.6363487153623809), (5092, 1.29943169...",26.001177,2.415990,4,28819.392920


In [6]:
len(colonies)

4290

In [18]:
colonies[colonies['USO_FINAL'] == 'JJC']

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,canal,railway,drain,barrier,nbrs_touch,nbrs_bbox,centroid,nbrs_dist_touch,nbrs_dist_bbox,ndmc_dist_km,area_km2,index,population
613,Jawahar Camp,1836,G,JJC,"POLYGON Z ((1013902.860 1002043.384 0.000, 101...",False,False,False,False,"[2032, 2895, 2215, 2709, 2708, 3340, 3390, 349...","[2032, 2895, 2215, 2709, 2708, 2048, 1124, 238...",POINT (1013282.680 1001576.776),"[(2032, 1.5663091213076403), (2895, 0.80133430...","[(2032, 1.5663091213076403), (2895, 0.80133430...",8.223415,0.547357,613,17102.976385
983,Lal Gumbad & JJC,2100,G,JJC,"POLYGON Z ((1021118.743 990436.401 0.000, 1021...",False,False,False,False,"[2828, 2536, 2182, 3504, 3717]","[2828, 2536, 2182, 3504, 3717]",POINT (1020964.642 990583.258),"[(2828, 0.2377070278312887), (2536, 0.46675725...","[(2828, 0.2377070278312887), (2536, 0.46675725...",10.307511,0.075404,983,3711.508757
1105,Jawaharlal Nehru Camp,1839,G,JJC,"POLYGON Z ((1025362.275 989667.318 0.000, 1025...",False,False,False,False,"[1687, 1596, 2407, 1914, 3766]","[1687, 1596, 2407, 1914, 3766]",POINT (1025493.377 989762.408),"[(1687, 0.2126083507697687), (1596, 0.12552396...","[(1687, 0.2126083507697687), (1596, 0.12552396...",11.819228,0.013768,1105,932.046818
1108,Navjeewan Camp,2407,G,JJC,MULTIPOLYGON Z (((1025306.770 989545.674 0.000...,False,False,False,False,"[1839, 1275, 1914, 3766]","[1839, 1275, 1914, 3568, 3766]",POINT (1025436.343 989588.245),"[(1839, 0.18326346735018562), (1275, 0.1993643...","[(1839, 0.18326346735018562), (1275, 0.1993643...",11.964171,0.002961,1108,4070.190876
1110,Bhoomi Heen Camp,1275,G,JJC,MULTIPOLYGON Z (((1025233.941 989323.086 0.000...,False,False,False,False,"[3186, 2407, 1914, 3568, 3766, 4805]","[3186, 2407, 1914, 3568, 3766, 4805, 5287]",POINT (1025325.295 989422.672),"[(3186, 0.9705017848326803), (2407, 0.19936431...","[(3186, 0.9705017848326803), (2407, 0.19936431...",12.084711,0.006567,1110,978.653342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798,Shaheed Arjun Dass Camp on Kushak Nalla Betwee...,4066,None,JJC,"POLYGON Z ((1019182.485 997685.262 0.000, 1019...",False,False,False,False,[],[],POINT (1019195.383 997676.849),[],[],3.930725,0.000234,2798,25.762316
2799,"Sant Ravi Dass Camp L-Block, Shakur Pur.",4067,None,JJC,"POLYGON Z ((1013799.139 1007170.090 0.000, 101...",False,False,False,False,[2800],"[2800, 2859, 2957]",POINT (1013819.240 1007184.973),"[(2800, 0.13954725849717983)]","[(2800, 0.13954725849717983), (2859, 0.2956053...",9.920111,0.001357,2799,421.160465
2800,Block-17 & 21 Kalyan Puri,4068,None,JJC,"POLYGON Z ((1030508.646 999065.118 0.000, 1030...",False,False,False,False,"[1916, 1045]","[1916, 1918, 1045]",POINT (1030463.304 999122.686),"[(1916, 0.2775677694698185), (1045, 0.29848578...","[(1916, 0.2775677694698185), (1918, 0.32758676...",9.156775,0.030372,2800,4176.718828
2801,Jhuggies Kholi Wala Baba Mandir Rajokari,4069,None,JJC,"POLYGON Z ((1012061.565 987627.523 0.000, 1012...",False,False,False,False,[2707],[2707],POINT (1012021.991 987524.514),"[(2707, 0.5708077573853915)]","[(2707, 0.5708077573853915)]",16.361608,0.004570,2801,102.645336


In [7]:
colonies_subset = colonies[colonies['area_km2'] > .009]

In [8]:
len(colonies_subset)

3480

In [10]:
colonies_subset[colonies_subset['USO_FINAL'] == 'JJC']

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,canal,railway,drain,barrier,nbrs_touch,nbrs_bbox,centroid,nbrs_dist_touch,nbrs_dist_bbox,ndmc_dist_km,area_km2,index,population
613,Jawahar Camp,1836,G,JJC,"POLYGON Z ((1013902.860 1002043.384 0.000, 101...",False,False,False,False,"[2032, 2895, 2215, 2709, 2708, 3340, 3390, 349...","[2032, 2895, 2215, 2709, 2708, 2048, 1124, 238...",POINT (1013282.680 1001576.776),"[(2032, 1.5663091213076403), (2895, 0.80133430...","[(2032, 1.5663091213076403), (2895, 0.80133430...",8.223415,0.547357,613,17102.976385
983,Lal Gumbad & JJC,2100,G,JJC,"POLYGON Z ((1021118.743 990436.401 0.000, 1021...",False,False,False,False,"[2828, 2536, 2182, 3504, 3717]","[2828, 2536, 2182, 3504, 3717]",POINT (1020964.642 990583.258),"[(2828, 0.2377070278312887), (2536, 0.46675725...","[(2828, 0.2377070278312887), (2536, 0.46675725...",10.307511,0.075404,983,3711.508757
1105,Jawaharlal Nehru Camp,1839,G,JJC,"POLYGON Z ((1025362.275 989667.318 0.000, 1025...",False,False,False,False,"[1687, 1596, 2407, 1914, 3766]","[1687, 1596, 2407, 1914, 3766]",POINT (1025493.377 989762.408),"[(1687, 0.2126083507697687), (1596, 0.12552396...","[(1687, 0.2126083507697687), (1596, 0.12552396...",11.819228,0.013768,1105,932.046818
1147,Transit Camp,3181,G,JJC,"POLYGON Z ((1026412.942 989596.106 0.000, 1026...",False,False,False,False,"[1595, 1914]","[1595, 1914, 3802]",POINT (1026323.603 989623.459),"[(1595, 0.6583191191131449), (1914, 0.52902986...","[(1595, 0.6583191191131449), (1914, 0.52902986...",12.254053,0.115881,1147,14969.343718
1401,"JJ Indra Camp , Saidullajaib",1872,G,JJC,"POLYGON Z ((1019738.644 988792.529 0.000, 1019...",False,False,False,False,[2849],[2849],POINT (1019632.423 988708.810),"[(2849, 0.8720821415834747)]","[(2849, 0.8720821415834747)]",12.308157,0.038698,1401,247.773410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2765,B-58 Rama Road,4033,None,JJC,"POLYGON Z ((1014901.493 1003894.372 0.000, 101...",False,False,False,False,[],[],POINT (1014992.466 1003818.900),[],[],7.119649,0.016591,2765,397.242852
2769,JJC at Goela Dairy Najafgarh,4037,None,JJC,"POLYGON Z ((1002112.937 994916.171 0.000, 1002...",False,False,False,False,[],"[3146, 2656, 4425]",POINT (1002252.545 994893.965),[],"[(3146, 1.4526840651914898), (2656, 0.62637017...",20.133663,0.024617,2769,272.419313
2771,LNJP Hospital Ranjeet Road,4039,None,JJC,"POLYGON Z ((1022734.079 1001575.504 0.000, 102...",False,False,False,False,[2814],"[2814, 3065]",POINT (1022689.721 1001636.284),"[(2814, 0.6593200717876416)]","[(2814, 0.6593200717876416), (3065, 0.35988499...",1.430788,0.011441,2771,363.034271
2783,Haryana Power House,4051,None,JJC,"POLYGON Z ((1014143.667 1005549.248 0.000, 101...",False,False,False,False,[],[],POINT (1014185.583 1005473.881),[],[],8.618409,0.018302,2783,381.769517


In [11]:
small_colonies = colonies[colonies['area_km2'] <= .009]

In [12]:
small_colonies.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,canal,railway,drain,barrier,nbrs_touch,nbrs_bbox,centroid,nbrs_dist_touch,nbrs_dist_bbox,ndmc_dist_km,area_km2,index,population
8,Sanjay Colony Narela,2869,G,UAC,MULTIPOLYGON Z (((1008702.105 1025447.403 0.00...,False,False,False,False,"[5105, 5436, 5457]","[5105, 5436, 5445, 5457]",POINT (1008750.283 1025645.257),"[(5105, 0.3690832846815001), (5436, 0.36908332...","[(5105, 0.3690832846815001), (5436, 0.36908332...",27.845413,0.006579,8,215.830459
240,Sardar Nagar near CC Colony,2900,D,Planned,"POLYGON Z ((1019057.952 1007175.956 0.000, 101...",False,False,False,False,"[1305, 2624]","[1305, 2624]",POINT (1019027.833 1007183.146),"[(1305, 0.11105755875592915), (2624, 0.0906456...","[(1305, 0.11105755875592915), (2624, 0.0906456...",6.763830,0.001965,240,297.481928
380,Sher Singh Enclave Karala,2995,G,UAC,MULTIPOLYGON Z (((1005491.397 1011144.496 0.00...,False,False,False,False,"[1804, 2701, 2736, 4770, 4776, 4900, 5169, 5178]","[1804, 2701, 2736, 4770, 4776, 4900, 5169, 5178]",POINT (1005395.410 1011101.204),"[(1804, 0.15969184481215065), (2701, 0.4642238...","[(1804, 0.15969184481215065), (2701, 0.4642238...",19.055434,0.005261,380,69.662397
416,Kucha Bulaki Begam,2068,E,SDA,"POLYGON Z ((1022959.344 1003095.131 0.000, 102...",False,False,False,False,"[2240, 2067, 1816, 1971, 1325]","[2240, 2067, 1816, 1971, 1325]",POINT (1022901.891 1003141.862),"[(2240, 0.15520276839526442), (2067, 0.1305203...","[(2240, 0.15520276839526442), (2067, 0.1305203...",2.675247,0.008555,416,494.836948
426,Katra Pedan,1974,E,SDA,"POLYGON Z ((1021556.577 1003667.664 0.000, 102...",False,False,False,False,"[1968, 2414, 2001]","[1968, 2414, 2001]",POINT (1021586.658 1003735.491),"[(1968, 0.18082914557537674), (2414, 0.0794064...","[(1968, 0.18082914557537674), (2414, 0.0794064...",2.859559,0.006348,426,127.248277


In [16]:
small_colonies.groupby('USO_FINAL').size()

USO_FINAL
JJC        573
JJR          2
Other        3
Planned     21
RUAC        18
RV           1
SDA          3
UAC        188
UV           1
dtype: int64

In [17]:
len(small_colonies)

810

## Load colonies dataset and create two versions

In [19]:
colonies_touch = colonies_subset.drop(columns=['nbrs_bbox', 'nbrs_dist_bbox'])

In [20]:
colonies_touch.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,canal,railway,drain,barrier,nbrs_touch,centroid,nbrs_dist_touch,ndmc_dist_km,area_km2,index,population
0,Singhola,3058,H,RV,"POLYGON Z ((1013763.588 1023721.838 0.000, 101...",False,False,True,True,[],POINT (1012643.931 1023967.383),[],24.721016,2.733999,0,4415.586042
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((1007997.730 1025421.961 0.000, 100...",False,True,False,True,"[2869, 5445]",POINT (1008331.501 1025234.188),"[(2869, 0.5868185653539563), (5445, 0.31827133...",27.676855,0.182393,1,3990.707763
2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((1008543.236 1022671.585 0.000, 100...",False,True,False,True,[5478],POINT (1009672.909 1022802.723),"[(5478, 0.661418841164336)]",24.900046,1.470808,2,6849.523714
3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((1008080.674 1025132.190 0.000, 100...",False,True,False,True,"[2869, 5095, 5105, 5436, 5457]",POINT (1008902.120 1025098.259),"[(2869, 0.5676799818012491), (5095, 0.32048263...",27.289733,1.150775,3,26712.165262
4,Kureni,2082,H,RV,"POLYGON Z ((1009508.695 1025281.671 0.000, 100...",False,True,False,True,"[3963, 5092, 5098, 5104, 5106, 5451]",POINT (1009491.379 1023952.320),"[(3963, 0.6363487153623809), (5092, 1.29943169...",26.001177,2.415990,4,28819.392920


In [21]:
colonies_bbox = colonies_subset.drop(columns=['nbrs_touch', 'nbrs_dist_touch'])

In [22]:
colonies_bbox.head()

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry,canal,railway,drain,barrier,nbrs_bbox,centroid,nbrs_dist_bbox,ndmc_dist_km,area_km2,index,population
0,Singhola,3058,H,RV,"POLYGON Z ((1013763.588 1023721.838 0.000, 101...",False,False,True,True,"[5108, 5487, 5488]",POINT (1012643.931 1023967.383),"[(5108, 1.4513506146004875), (5487, 1.17056023...",24.721016,2.733999,0,4415.586042
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((1007997.730 1025421.961 0.000, 100...",False,True,False,True,"[2869, 5445]",POINT (1008331.501 1025234.188),"[(2869, 0.5868185653539563), (5445, 0.31827133...",27.676855,0.182393,1,3990.707763
2,Bhor Garh,1276,H,Industrial,"POLYGON Z ((1008543.236 1022671.585 0.000, 100...",False,True,False,True,"[4270, 4273, 5478]",POINT (1009672.909 1022802.723),"[(4270, 0.8033104667179346), (4273, 0.58109669...",24.900046,1.470808,2,6849.523714
3,Gautam Colony,1528,G,RUAC,"POLYGON Z ((1008080.674 1025132.190 0.000, 100...",False,True,False,True,"[2869, 3963, 4243, 5092, 5095, 5105, 5436, 544...",POINT (1008902.120 1025098.259),"[(2869, 0.5676799818012491), (3963, 0.70232814...",27.289733,1.150775,3,26712.165262
4,Kureni,2082,H,RV,"POLYGON Z ((1009508.695 1025281.671 0.000, 100...",False,True,False,True,"[3963, 5092, 5095, 5098, 5104, 5106, 5451, 5478]",POINT (1009491.379 1023952.320),"[(3963, 0.6363487153623809), (5092, 1.29943169...",26.001177,2.415990,4,28819.392920


## Import services shapefiles

In [23]:
# Define filepaths

services_dir = os.path.join('shapefiles', 'Spatial_Index_GIS', 'Public Services')

bank_fp = os.path.join(services_dir, 'Banking', 'Banking.shp')
health_fp = os.path.join(services_dir, 'Health', 'Health.shp')
road_fp = os.path.join(services_dir, 'Major Road', 'Road.shp')
police_fp = os.path.join(services_dir, 'Police', 'Police Station.shp')
ration_fp = os.path.join(services_dir, 'Ration', 'Ration.shp')
school_fp = os.path.join(services_dir, 'School', 'schools7760.shp')
transport_fp = os.path.join(services_dir, 'Transport', 'Transport.shp')

# boundary of Delhi
delhi_bounds_filepath = os.path.join('shapefiles', 'delhi_bounds_buffer.shp')

# Check that all filepaths exist
filepath_list = [bank_fp, health_fp, road_fp, police_fp, ration_fp, school_fp, transport_fp, delhi_bounds_filepath]

for filepath in filepath_list:
    if not os.path.exists(filepath):
        print('{} does not exist'.format(filepath))

In [24]:
# Import services
bank = gpd.read_file(bank_fp)
health = gpd.read_file(health_fp)
road = gpd.read_file(road_fp)
police = gpd.read_file(police_fp)
ration = gpd.read_file(ration_fp)
school = gpd.read_file(school_fp)
transport = gpd.read_file(transport_fp)

## Check validity of services shapefiles
* Duplicate rows are okay for ATMs (I assume that ATM locations for the same bank in a similar location will seem to be counted twice)
* Look specifically for invalid geometries and whether shapefile is fully contained within Delhi

In [ ]:
spatial_index_utils.check_shapefile(gdf=bank, gdf_name='bank', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

In [ ]:
spatial_index_utils.check_shapefile(gdf=health, gdf_name='health', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

In [ ]:
spatial_index_utils.check_shapefile(gdf=road, gdf_name='road', 
                                    geom_type='Line', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

In [ ]:
spatial_index_utils.check_shapefile(gdf=police, gdf_name='police', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

In [ ]:
spatial_index_utils.check_shapefile(gdf=ration, gdf_name='ration', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

In [ ]:
spatial_index_utils.check_shapefile(gdf=school, gdf_name='school', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

In [ ]:
spatial_index_utils.check_shapefile(gdf=transport, gdf_name='transport', 
                                    geom_type='Point', 
                                    delhi_bounds_filepath=delhi_bounds_filepath)

## Check CRS (all shapefiles should be in EPSG: 7760)

In [25]:
bank.crs == health.crs == road.crs == police.crs == ration.crs == school.crs == transport.crs

True

In [ ]:
bank.crs

In [26]:
colonies_bbox.crs == colonies_touch.crs == bank.crs

True

## Define Point and Line Services

In [27]:
# Define all point services as dictionary
# makes it easier to calculate all point
# services with one function
point_services = {'bank': bank,
                  'health': health,
                  'police': police,
                  'ration': ration,
                  'school': school,
                  'transport': transport}

line_services = {'road': road}

## Calculate all service indices in one function

In [28]:
from spatial_index_utils import calc_all_services

In [29]:
colonies_touch_psi = calc_all_services(colonies_touch, point_services, line_services, epsg_code, 'nbrs_dist_touch')

GeoDataFrame now has the following CRS:

epsg:7760


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
colonies_touch_psi = colonies_touch_psi.rename(columns={'road_count':'road_length'})
colonies_touch_psi

In [ ]:
colonies_bbox_psi = calc_all_services(colonies_bbox, point_services, line_services, epsg_code, 'nbrs_dist_bbox')

In [ ]:
colonies_bbox_psi = colonies_bbox_psi.rename(columns={'road_count':'road_length'})
colonies_bbox_psi.head()

## Save Files

In [ ]:
colonies_bbox_psi.drop(columns=['nbrs_bbox', 'nbrs_dist_bbox', 'centroid']).to_file('delhi_psi_bbox_11Aug2020.shp')

In [ ]:
colonies_touch_psi.drop(columns=['nbrs_touch', 'nbrs_dist_touch', 'centroid']).to_file('delhi_psi_touch_11Aug2020.shp')

In [ ]:
colonies_bbox_psi.to_csv('delhi_psi_bbox_11Aug2020.csv')

In [ ]:
colonies_touch_psi.to_csv('delhi_psi_touch_11Aug2020.csv')

In [ ]:
with open('colonies_bbox_psi.data', 'wb') as f:
    pickle.dump(colonies_bbox_psi, f)

In [ ]:
with open('colonies_touch_psi.data', 'wb') as f:
    pickle.dump(colonies_touch_psi, f)